In [ ]:
import duckdb
import pandas as pd
import folium

In [ ]:
HEALTH_DB = "../data/healthkit-sqlite-2023-11-17.db"

In [ ]:
con = duckdb.connect(HEALTH_DB)

In [ ]:
con.install_extension("sqlite")

In [ ]:
con.load_extension("sqlite")

In [ ]:
con.sql("PRAGMA show_tables")

In [ ]:
camino_sql = "SELECT * from workouts WHERE sourceName != 'AllTrails' AND startDate >= '2023-10-10' AND startDate <= '2023-10-17' AND duration > 100"

In [ ]:
camino_bilbao_sql = "SELECT * from workouts WHERE sourceName != 'AllTrails' AND startDate >= '2023-10-16' AND startDate <= '2023-10-17' AND duration < 100"

In [ ]:
workouts_df = con.sql(camino_sql).to_df()

In [ ]:
bilbao_df = con.sql(camino_bilbao_sql).to_df()

In [ ]:
bilbao_df

In [ ]:
workouts_df.info()

In [ ]:
workouts_df.columns

In [ ]:
drop_cols = [
    "workoutActivityType",
    "durationUnit",
    "sourceVersion",
    "metadata_HKGroupFitness",
    "metadata_HKWorkoutBrandName",
    "metadata_HKTimeZone",
    "metadata_HKCoachedWorkout",
    "metadata_HKWasUserEntered",
    "workout_events",
    "metadata_HKWeatherTemperature",
    "metadata_HKWeatherHumidity",
    "metadata_HKIndoorWorkout",
    "metadata_HKElevationAscended",
    "metadata_HKSwimmingLocationType",
    "metadata_HKAverageMETs",
    "metadata_HEALTHFIT_SUB_SPORT",
    "metadata_HEALTHFIT_ROUTE",
    "metadata_HEALTHFIT_FILE_TYPE",
    "metadata_HKMaximumSpeed",
    "metadata_HEALTHFIT_TOTAL_MOVING_TIME",
    "metadata_HEALTHFIT_TOTAL_DISTANCE",
    "metadata_HEALTHFIT_MAX_RUNNING_CADENCE",
    "metadata_HEALTHFIT_MIN_ALTITUDE",
    "metadata_HEALTHFIT_AVG_RUNNING_CADENCE",
    "metadata_HEALTHFIT_APP_BUILD",
    "metadata_HEALTHFIT_FIT_SPORT",
    "metadata_HEALTHFIT_FIT_SUB_SPORT",
    "metadata_HEALTHFIT_MAX_ALTITUDE",
    "metadata_HEALTHFIT_FIT_MANUFACTURER",
    "metadata_HEALTHFIT_TOTAL_STRIDES",
    "metadata_HEALTHFIT_FIT_SERIAL_NUMBER",
    "metadata_HEALTHFIT_SPORT",
    "metadata_HKAverageSpeed",
    "metadata_HEALTHFIT_APP_VERSION",
    "metadata_HKExternalUUID",
    "metadata_HKElevationDescended",
    "metadata_HKLapLength",
]

In [ ]:
workouts_df.drop(labels=drop_cols, axis=1, inplace=True)

In [ ]:
workouts_df

In [ ]:
walk_id = con.sql(camino_sql.replace("*", "id")).to_df()

In [ ]:
walk_ids = walk_id["id"].values.tolist()

In [ ]:
walk_ids.pop(0)  # Evening walk in San Sebastin

In [ ]:
bilbao_df["id"].iloc[0]

In [ ]:
walk_ids.append(bilbao_df["id"].iloc[0])

In [ ]:
walk_sql = "SELECT * FROM workout_points WHERE workout_id = 'WORKOUT_ID'"

In [ ]:
m = folium.Map(location=[43.3183, -1.9812], zoom_start=12, tiles="openstreetmap")

In [ ]:
def update_map(m, df):
    points = df[["latitude", "longitude"]].values.tolist()
    folium.PolyLine(points, color="blue", weight=3.5, opacity=1).add_to(m)
    folium.Marker([df["latitude"].iloc[0], df["longitude"].iloc[0]]).add_to(m)
    return m

In [ ]:
for id in walk_ids:
    # TODO: Adjust SQL to only get Camino walks (plus extra in Bilbao) - DONE
    print(f"Getting data for walk ID: {id}")
    walk_df = con.sql(walk_sql.replace("WORKOUT_ID", id)).to_df()
    # TODO: Cache / preprocess the walk data
    m = update_map(m, walk_df)
m.fit_bounds(m.get_bounds())

In [ ]:
m